### Data munging (General)

#### Importing libraries and source files

In [334]:

import xlwings as xw
import os
import pandas as pd
import numpy as np
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [335]:

#-- Batch 5A defined
batch5a = 'FSC-JAD-01-TEC-61-A1 MV FMECA-Batch 5 LBHD - AN.xlsx' 
pathBatch5a = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets'
shtBatch5a = '5- 4S WBT FMECA'
cellRangeBatch5a = 'A5:BQ514'


Assigne the above variables to the ones used in this workbook

In [336]:
nameFile = batch5a
path = pathBatch5a
sheet = shtBatch5a
cellRange = cellRangeBatch5a

In [337]:
file_path = os.path.join(path,nameFile)
book = xw.Book(file_path)
sht = book.sheets[sheet]
rng = sht.range(cellRange)
df = rng.options(pd.DataFrame, index=False, header=True).value

In [338]:
df.head(1)


,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Column1,Assessment notes,Adj Final action,Coating condition
0,1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,None,None,Local Generalised corrosion on the longitudina...,None,Poor


#### Stiffener name correction

The stiffeners listed in the Vertech reports have some inconsistencies. <br>


Some appear as LL06-07 <br>

    In this case it will be replaced with the "nearest longitudinal stiffener". 
    It should be either one the above.
    
Some appear as HG1, HG2 etc. <br>

    HG1, HG2, HG3 are also the same as LL09, LL16, and LL22 respectively.
    They will be replaced with the stiffener names.

This will create a new column. <br>

The column should be coppied and replaced with the "stiffener" column in the FMECA sheet.


In [339]:
def stiffenerCorrection(row):
    stiffener = row['Stiffener(s)']
    nearest_lng = row['Nearest longitudinal member (SS,ObLBHD)']
    if stiffener == 'HG1':
        return 'LL09'
    elif stiffener == 'HG2':
        return 'LL16'
    elif stiffener == 'HG3':
        return 'LL22'
    elif len(stiffener.split('-')) == 2:
        return nearest_lng
    else:
        return stiffener

In [340]:
df['new_stiffener'] = df.apply(stiffenerCorrection,axis=1)

In [341]:
df.head(1)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Column1,Assessment notes,Adj Final action,Coating condition,new_stiffener
0,1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,None,None,Local Generalised corrosion on the longitudina...,None,Poor,LL15


#### As-built thickness correction

In [342]:
abt_lbhd = {"LL00":14,
"LL01":14,
"LL02":14,
"LL03":14,
"LL04":11.5,
"LL05":11.5,
"LL06":11.5,
"LL07":11.5,
"LL08":12,
"LL09":12,
"LL10":12,
"LL11":12,
"LL12":13,
"LL13":13,
"LL14":13,
"LL15":13,
"LL16":14.5,
"LL17":14.5,
"LL18":14.5,
"LL19":14.5,
"LL20":15,
"LL21":15,
"LL22":15,
"LL23":16,
"LL24":16,
"LL25":16,
"LL26":19.5,
"LL27":19.5,
"UD":20.5,
"BP":18.0
}

In [343]:
abt_web = {"LL00":10,
"LL01":10,
"LL02":10,
"LL03":10,
"LL04":10,
"LL05":10,
"LL06":10,
"LL07":10,
"LL08":10,
"LL09":10,
"LL10":10,
"LL11":10,
"LL12":10,
"LL13":10,
"LL14":10,
"LL15":10,
"LL16":10,
"LL17":10,
"LL18":10.5,
"LL19":10.5,
"LL20":10.5,
"LL21":10.5,
"LL22":10.5,
"LL23":10.5,
"LL24":10.5,
"LL25":10.5,
"LL26":10.5,
"LL27":10.5,
"UD":12.0,
"BP":13.5
}


In [344]:
abt_flange = {"LL00":15.5,
"LL01":15.5,
"LL02":17,
"LL03":17,
"LL04":18.5,
"LL05":18.5,
"LL06":17,
"LL07":17,
"LL08":20,
"LL09":20,
"LL10":20,
"LL11":20,
"LL12":20,
"LL13":20,
"LL14":22,
"LL15":22,
"LL16":19.5,
"LL17":19.5,
"LL18":18,
"LL19":18,
"LL20":20,
"LL21":21,
"LL22":23,
"LL23":23,
"LL24":19.5,
"LL25":20.5,
"LL26":24.5,
"LL27":14,
"UD":17,
"BP":25
}

In [345]:
def abt_check(row):
    
    if row['Local Structure'] == 'Plate':
        if row['Stiffener(s)'] in abt_lbhd:
            if abt_lbhd[row['Stiffener(s)']] == row["As Built Thickness\n(mm)"]:
                return 'abt correct'
            else:
                return 'check abt'
        else:
            return 'find abt from drawings'
    elif row['Local Structure'] == "Long'l stiff'r":
        if row['Detail Structure'] == 'Web':
            if row['Stiffener(s)'] in abt_web:
                if abt_web[row['Stiffener(s)']] == row["As Built Thickness\n(mm)"]:
                    return 'abt correct'
                else:
                    return 'check abt'
            else:
                return 'find abt from drawings'
        elif row['Detail Structure'] == 'Flange':
            if row['Stiffener(s)'] in abt_flange:
                if abt_flange[row['Stiffener(s)']] == row["As Built Thickness\n(mm)"]:
                    return 'abt correct'
                else:
                    return 'check abt'
            else:
                return 'find abt from drawings'

        else:
            return 'not a web/flange'
    else:
        return 'not a plate/stiffener'
        

In [346]:
df['abtCheck'] = df.apply(abt_check, axis=1)

In [347]:
def correct_abt(row):
    vertech_abt = row["As Built Thickness\n(mm)"]
    if row['Local Structure'] == 'Plate':
        if row['abtCheck'] == 'check abt':
            return abt_lbhd[row['Stiffener(s)']]
        else:
            return vertech_abt
    elif row['Local Structure'] == "Long'l stiff'r":
        if row['Detail Structure'] == 'Web':
            if row['abtCheck'] == 'check abt':
                return abt_web[row['Stiffener(s)']]
            else:
                return vertech_abt
        elif row['Detail Structure'] == 'Flange':
            if row['abtCheck'] == 'check abt':
                return abt_flange[row['Stiffener(s)']]
            else:
                return vertech_abt
    else:
        return vertech_abt
        

In [348]:

df['correctAbt'] = df.apply(correct_abt, axis=1)

In [349]:
correct_abts = df[['Anomaly ID','Stiffener(s)','As Built Thickness\n(mm)','Local Structure','Detail Structure','abtCheck','correctAbt','new_stiffener']]

In [350]:
correct_abts.shape

(509, 8)

In order to view the table uncommate the line below.

In [351]:
#xw.view(correct_abts)

#### Function for Final Anomaly Structure

In [352]:
def finalAnomalyDescription(row):
    if row['Final Anomaly description'] == 'Pit within generalised corrosion':
        return 'Pit within generalised corrosion'
    else:
        if row['Anomaly Nature'] == 'Corrosion':
            return 'Generalised corrosion'
        elif row['Anomaly Nature'] == 'Pitting':
            return 'Pit'
        else:
            return ''

In [353]:
df['final_anomlay_description'] = df.apply(finalAnomalyDescription,axis=1)

#### Function for "location" structure

In [354]:
def locationStructure(row):
    if row['Local Structure'] == 'Plate':
        return 'Tank boundary plating'
    elif row['Local Structure'] == "Long'l stiff'r":
        if row['Detail Structure'] == 'Web':
            return 'Stiffener web'
        elif row['Detail Structure'] == 'Flange':
            return 'Stiffener flange middle 1/3'
    else:
        return ''

In [355]:
df['location_formulated'] = df.apply(locationStructure,axis=1)

In [356]:
#xw.view(df)

#### Stop!
Copy and paste the above newly calculated columns into the original fmeca sheet.

Then re-start from the beggining and then apply the functions below

#### Function for "detail" status

In [357]:
def detailStatus(row):
    remaining_t = row['Average UTM Reading (mm)']
    loss = row['(mm)']
    renewable_loss = row["As Built Thickness\n(mm)"] - row["Renewal Thickness (mm)"]
    substantial_loss = row["As Built Thickness\n(mm)"] - row["Substantial Corer Thickness (mm)"]
    flag = row['FLAG']

    if row['Final Anomaly description'] == 'Pit':
        if row['Location'] == 'Tank boundary plating':
            if remaining_t > 10.0:
                if flag == "R":
                    return 'More than 10mm remaining and greater than renewable loss'
                else:
                    return "Substantial or below substantial loss"
            elif 6.0 <= remaining_t and  remaining_t <= 10.0:
                if flag == "R":
                    return 'Between 6 - 10mm remaining'
                else:
                    return "Substantial or below substantial loss"
            elif remaining_t < 6.0:
                return 'Less than 6mm remaining'
            else:
                return 'investigate'
        if row['Location'] == 'Stiffener web':
            if loss > renewable_loss:
                if remaining_t < 6.0:
                    return 'Any loss greater than renewable (<6mm)'
                else:
                    return 'Any loss greater than renewable'
            else:
                return "Substantial or below substantial loss"
        if row['Location'] == 'Stiffener flange middle 1/3':
            if loss > renewable_loss:
                if remaining_t < 6.0:
                    return 'Any loss greater than renewable (<6mm)'
                else:
                    return 'Any loss greater than renewable'
            else:
                return "Substantial or below substantial loss"
        if row['Location'] == 'Stiffener flange outside of middle 1/3':
            if remaining_t >= 6.0:
                if flag == "R":
                    return 'More than 6mm remaining and greater than renewable loss'
                else:
                    return "Substantial or below substantial loss"
            elif remaining_t < 6.0:
                return 'Less than 6mm remaining'
    if row['Final Anomaly description'] == 'Generalised corrosion':
        if remaining_t < 6.0:
            return 'Less than 6mm remaining'
        else:
            return "Greater than 6mm remaining"
    if row['Final Anomaly description'] == 'Pit within generalised corrosion':
        if remaining_t < 6.0:
            return 'Less than 6mm remaining'
        else:
            return "Greater than 6mm remaining"


    

In [358]:
df['detail_formulated'] = df.apply(detailStatus,axis=1)

In [359]:
#xw.view(df)

#### Function for "Repair method"

In [360]:
def RepairMethodR1(row):
    detail = row['Detail']
    coating = row['Coating condition']
    anomaly_type = row['Final Anomaly description']
    location = row['Location']

    if coating != "":
        if anomaly_type == 'Pit':
            if location == 'Tank boundary plating':
                if detail == 'More than 10mm remaining and greater than renewable loss' or detail == "Substantial or below substantial loss":
                    if coating == 'Fair' or coating == 'Poor':
                        return "Arrest and monitor CVI"
                    else:
                        return "Monitor only, no coating"
                elif detail == 'Between 6 - 10mm remaining':
                    return 'Arrest and monitor CVI'
                elif detail == 'Less than 6mm remaining':
                    return 'Permanent repair'
                elif detail == "Substantial or below substantial loss":
                    if coating == 'Fair' or coating == 'Poor':
                        return "Arrest and monitor CVI"
                    else:
                        return "Monitor only, no coating"
                else:
                    return 'Pit. not above 10 or below 10?'

            elif location == 'Stiffener web':
                if detail == 'Any loss greater than renewable':
                    return "Arrest and monitor CVI"
                elif detail == 'Any loss greater than renewable (<6mm)':
                    return 'Permanent repair'
                else:
                    if coating == 'Fair' or coating == 'Poor':
                        return "Arrest and monitor CVI"
                    else:
                        return "Monitor only, no coating"
            elif location == 'Stiffener flange middle 1/3':
                if detail == 'Any loss greater than renewable':
                    return "Arrest and monitor CVI"
                elif detail == 'Any loss greater than renewable (<6mm)':
                    return 'Permanent repair'
                else:
                    if coating == 'Fair' or coating == 'Poor':
                        return "Arrest and monitor CVI"
                    else:
                        return "Monitor only, no coating"
            elif location == 'Stiffener flange outside of middle 1/3':
                if detail == 'More than 6mm remaining and greater than renewable loss':
                    return 'Permanent repair'
                elif detail == 'Less than 6mm remaining':
                    return 'Permanent repair'
        if anomaly_type == 'Generalised corrosion':
            
            if detail == 'Less than 6mm remaining':
                return 'Structural repair'
            elif detail == "Greater than 6mm remaining":
                if coating == 'Fair' or coating == 'Poor':
                    return "Arrest and monitor CVI"
                else:
                    return "Monitor only (not part of standard repair)"
            else:
                return 'Corr. not above 10, or below 10?'
        if anomaly_type == 'Pit within generalised corrosion':
            if detail == 'Less than 6mm remaining':
                return 'Structural repair'
            elif detail == "Greater than 6mm remaining":
                if coating == 'Fair' or coating == 'Poor':
                    return "Arrest and monitor CVI"
                else:
                    return "Monitor only (not part of standard repair)"
            else:
                return 'Corr. not above 10, or below 10?'
    else:
        return 'define coating condition'


In [361]:
df.head(1)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Column1,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,detail_formulated
0,1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,None,None,Local Generalised corrosion on the longitudina...,None,Poor,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining


In [362]:
df['repair_formulate'] = df.apply(RepairMethodR1,axis=1)

In [363]:
xw.view(df)

#### Importing diminution percentage table & Decision matrix

In [365]:
dm = 'decisionMatrix.xlsx' 
pathANworking = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets/AN_working'
decisionMatrixPath = os.path.join(pathANworking,dm)

In [366]:
dimLevels = pd.read_excel(decisionMatrixPath,sheet_name='diminutionLevels')

In [367]:
dm = pd.read_excel(decisionMatrixPath,sheet_name='decisionMatrix')

In [368]:
dimLevels

,Primary structure,Local structure,Diminution level
0,Long'l bhd,Plate,0.20
1,Long'l bhd,Long'l stiff'r,0.25


In [369]:
dm.head(1)

,Anomaly type,Location,Detail,Repair,Inspection interval
0,Pit,Tank boundary plating,More than 10mm remaining and greater than rene...,Permanent repair,Scheduled tank inspection


In [370]:
def inspectionInterval(row):
    anomaly_type = row['Final Anomaly description']
    location = row['Location']
    detail = row['Detail']
    repair = row['Repair']

    if anomaly_type == 'Pit':
        a = dm[dm['Anomaly type']==anomaly_type]
        b = a[a['Location']==location]
        c = b[b['Detail']==detail]
        d = c[c['Repair'] == repair]
        try:
            interval = d.iloc[0,-1]
        except:
            interval = 'error'
    else:
        b = dm[dm['Anomaly type']==anomaly_type]
        c = b[b['Detail']==detail]
        d = c[c['Repair'] == repair]
        try:
            interval = d.iloc[0,-1]
        except:
            interval = 'error'

    return interval
    
    



#### Function for inspection interval

In [371]:
df['inspInt_formulate'] = df.apply(inspectionInterval,axis=1)

In [372]:
def allowablePercentage(row):

    primary_structure = row["Primary Structure"]
    local_structure = row["Local Structure"]

    a = dimLevels[dimLevels['Primary structure']==primary_structure]
    b = a[a['Local structure']==local_structure]
    try:
        allowable = b.iloc[0,-1]
    except:
        allowable = float(0)

    return allowable


#### Allowable diminution percentage

In [373]:
def allowablechecks(row):
    dimLevelVertech = float(row["Max Allowable Diminution (%)"])
    dimLevelFSC = float(row["allowables"])
    diff = np.abs(dimLevelVertech-dimLevelFSC)
    if diff == 0:
        return 'correct dim level'
    else:
        return 'check dim level'



In [374]:
df['allowables'] = df.apply(allowablePercentage,axis=1)

In [375]:
df['allowableCheck'] = df.apply(allowablechecks,axis=1)

#### A summary fmeca sheet for view

In [376]:
summary_df_cols = ['Report reference', 'Anomaly ID','Stiffener(s)','Nearest longitudinal member (SS,ObLBHD)',
       'As Built Thickness\n(mm)', 
       'Renewal Thickness (mm)', 'Substantial Corer Thickness (mm)',
       'Average UTM Reading (mm)', 'Minimum \nUTT \nReading ', '(mm)', '(%)',
       'FLAG','correctAbt',
       'final_anomlay_description',
       'location_formulated', 'detail_formulated', 'repair_formulate']

In [377]:
summary_allowable = ['Anomaly ID','Stiffener(s)','Max Allowable Diminution (%)','Primary Structure','Local Structure','allowables','allowableCheck']

In [378]:
correct_dim = df[summary_allowable]

In [379]:
#xw.view(df[summary_allowable])

#### Function for notes


In [380]:
def notes(row):
    if row['Final Anomaly description'] != 'Pit within generalised corrosion':
        anomaly_type = 'Local '+ str(row['Final Anomaly description'])
    else:
        anomaly_type = row['Final Anomaly description']
    
    if row['Primary Structure'] == "Long'l bhd":
        structural_component = 'longitudinal bulkhead'

    if row["Local Structure"] == "Plate":
        local_component = 'plate'
    elif row["Local Structure"] == "Long'l stiff'r":
        local_component = "stiffener"
    else:
        local_component = ''

    if row["Detail Structure"] is None:
        detail_structure = ''
    else:
        detail_structure = row["Detail Structure"]
    coating_condition = row["Coating condition"].lower()

    if row["FLAG"] == 'R':
        flag = 'has reached renewal'
        calcs = 'calcs'
    elif row["FLAG"] == 'SC':
        flag = 'has reached substantial (not renewal)'
        calcs = 'no calcs'
    else:
        flag = 'is not at a critical'
        calcs = 'no calcs'
    remaining_t = str(row["Average UTM Reading (mm)"])

    
    if row['Final Anomaly description'] == 'Pit' and row["Average UTM Reading (mm)"] < 6.0:
         notes_text = f'{anomaly_type} found on the {structural_component} {local_component} with {coating_condition} coating. Remaining thickness {remaining_t}mm.'
    else:
        notes_text = f'{anomaly_type} on the {structural_component} {local_component} {detail_structure}. Average UT {flag} limit therefor {calcs} required. Coating condition is {coating_condition}.'
    notes_text = notes_text.replace(' .','.')
    return notes_text

In [381]:
df['notes_formulated'] = df.apply(notes,axis=1)

In [ ]:
#xw.view(df)

#### Function for "calc required Y/N"

In [385]:
def calcReqYN(row):
    final_anom_description = row['Final Anomaly description']
    flag = row['FLAG']

    
    if (final_anom_description == 'Generalised corrosion' or final_anom_description == 'Generalised corrosion') and flag == 'R':
        return 'Y'
    else:
        return 'N'

In [386]:
df['calc_y_n_formulate'] = df.apply(calcReqYN,axis=1)

In [387]:
xw.view(df)

### Calculations required Anomalies/stiffeners

#### Importing stiffener names and dimensions (standard)

In [388]:
#-- import the dataset containing stiffener dimensions
stiff_dim = pd.read_excel(decisionMatrixPath,sheet_name='Stiffeners')

In [389]:
# -- stiffeners
stiff_dim['Stiffener name'].unique()

array(['LL00', 'LL01', 'LL02', 'LL03', 'LL04', 'LL05', 'LL06', 'LL07',
       'LL08', 'LL09', 'LL10', 'LL11', 'LL12', 'LL13', 'LL14', 'LL15',
       'LL16', 'LL17', 'LL18', 'LL19', 'LL20', 'LL21', 'LL22', 'LL23',
       'LL24', 'LL25', 'LL26', 'LL27', 'BP', 'UD'], dtype=object)

In [390]:
stiff_dim[stiff_dim['Stiffener name']=='LL19'].iloc[0,1]

'500x10.5 + 150x18 L2'

#### Calculating columns required for the calculation sheet

In [391]:
#-- Filtering a dataset that requires calculations
req_calc = df[~(df['Final Anomaly description']=='Pit') & (df['FLAG']=='R')]

In [392]:
# -- check for null items where calculations are required
req_calc[req_calc["Stiffener(s)"].isnull()]

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Column1,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,detail_formulated,repair_formulate,inspInt_formulate,allowables,allowableCheck,notes_formulated,calc_y_n_formulate


In [393]:
# --  Check if there is any naming inconsistencies by comparing with the standard stiffener names
set(set(req_calc['Stiffener(s)'])).difference(stiff_dim['Stiffener name'])

set()

In [394]:
def stiff_dimensions(row,col_number):
    '''Takes the standard stiffener dimensions assign it to the corresponding stiffeners in the calc
    required dataset'''
    stiffener = row['Stiffener(s)']
    filter_dim = stiff_dim[stiff_dim['Stiffener name']==str(stiffener)].iloc[0,col_number]
    return filter_dim

In [395]:
def concatStiffners(row):
    '''Joining stiffener names with anomaly number
    '''
    return row['Stiffener(s)'] + "_" + row['Anomaly ID']

In [396]:
# -- Reseting index before calculation
req_calc.reset_index(inplace=True,drop=True)

In [397]:
req_calc.head(1)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Column1,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,detail_formulated,repair_formulate,inspInt_formulate,allowables,allowableCheck,notes_formulated,calc_y_n_formulate
0,1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,None,None,Local Generalised corrosion on the longitudina...,None,Poor,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,-,0.2,correct dim level,Local Generalised corrosion on the longitudina...,Y


In [398]:
# -- Applying stiffener dimension column
req_calc.loc[:,'stiffener_dim'] = req_calc.apply(stiff_dimensions,args = (1,),axis=1)


C:\Users\nisha\AppData\Local\Temp\ipykernel_18504\1917217339.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc.loc[:,'stiffener_dim'] = req_calc.apply(stiff_dimensions,args = (1,),axis=1)


In [399]:
req_calc['h'] = req_calc.apply(stiff_dimensions,args = (2,),axis=1)


C:\Users\nisha\AppData\Local\Temp\ipykernel_18504\1320414672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc['h'] = req_calc.apply(stiff_dimensions,args = (2,),axis=1)


In [400]:
req_calc['grade'] = req_calc.apply(stiff_dimensions,args = (3,),axis=1)
req_calc['ps'] = req_calc.apply(stiff_dimensions,args = (4,),axis=1)


C:\Users\nisha\AppData\Local\Temp\ipykernel_18504\1251418026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc['grade'] = req_calc.apply(stiff_dimensions,args = (3,),axis=1)
C:\Users\nisha\AppData\Local\Temp\ipykernel_18504\1251418026.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc['ps'] = req_calc.apply(stiff_dimensions,args = (4,),axis=1)


In [401]:
req_calc['stiff_anom_combo'] = req_calc.apply(concatStiffners,axis=1)

C:\Users\nisha\AppData\Local\Temp\ipykernel_18504\1422889332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc['stiff_anom_combo'] = req_calc.apply(concatStiffners,axis=1)


### Adjusting the thickness
The remaining thicknesses corresponds the thickness of the plate, flange or web based on where the anomaly is located. <br>
In the calculation sheet these three thicknesses are in three different columns. <br>
Therefore they need to be stored in three columns. <br>
The remaining thickness has to be decided based on the local structure

In [402]:
def thickness(row,local_structure,detail_structure):

    remaining_t = row["Average UTM Reading (mm)"]
    if row["Local Structure"] == local_structure:
        if row["Detail Structure"] == detail_structure:
            return remaining_t
    else:
        return ''

In [403]:
req_calc['tp'] = df.apply(thickness, args=('Plate',None),axis=1)

C:\Users\nisha\AppData\Local\Temp\ipykernel_18504\4171448176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc['tp'] = df.apply(thickness, args=('Plate',None),axis=1)


In [404]:
req_calc['tw'] = df.apply(thickness, args=("Long'l stiff'r","Web"),axis=1)

C:\Users\nisha\AppData\Local\Temp\ipykernel_18504\1944791771.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc['tw'] = df.apply(thickness, args=("Long'l stiff'r","Web"),axis=1)


In [405]:
req_calc['tf'] = df.apply(thickness, args=("Long'l stiff'r","Flange"),axis=1)

C:\Users\nisha\AppData\Local\Temp\ipykernel_18504\2064931562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_calc['tf'] = df.apply(thickness, args=("Long'l stiff'r","Flange"),axis=1)


#### A temporary excel file to print calculation sheet data

Further operations are done in the "jad_fmeca_b5_operation_calc.ipynb" sheet. <br>
Copying data from that file into the calculation file may be more useful. <br>
The export file will be used to feed the row items into the calculation sheet. <br>

In [406]:
req_calc.head(1)

,Item No.,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Column1,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,detail_formulated,repair_formulate,inspInt_formulate,allowables,allowableCheck,notes_formulated,calc_y_n_formulate,stiffener_dim,h,grade,ps,stiff_anom_combo,tp,tw,tf
0,1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,None,None,Local Generalised corrosion on the longitudina...,None,Poor,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,-,0.2,correct dim level,Local Generalised corrosion on the longitudina...,Y,400x10 + 150x22 L2,11.13,AH36,Sec,LL15_4SWBT-LBHD- CR-192,6.79,,


In [407]:
req_calc_cols = ["Stiffener(s)","stiff_anom_combo","stiffener_dim","h","grade","ps","Average UTM Reading (mm)","tp","tw","tf",
"Nearest transverse member (Fr., TWF,TBHD)", "Longitudinal\n(l) (distance\nfrom in mm)","Frame(s)","Primary Structure","Final Anomaly description","Location","Detail"]

In [408]:
reqCalc = req_calc[req_calc_cols]

In [409]:
reqCalc.sort_values(by=["Stiffener(s)","stiff_anom_combo"],ascending=True,inplace=True)

C:\Users\nisha\AppData\Local\Temp\ipykernel_18504\575848562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reqCalc.sort_values(by=["Stiffener(s)","stiff_anom_combo"],ascending=True,inplace=True)


### Saving Files

Save the the fmeca with columns that are calculated throughout this script

In [410]:
# --  Save names
extended_df_name = 'extended_fmeca.xlsx' # "df" ---- The whole fmeca - which has all the newly calculated columns
req_calc_name = 'fmeca_req_calc.xlsx' # "reqCalc" ---- A slice of the fmeca with selected columns that require calculation. The order of columns is important here.
correct_abts_name = 'correct_abts.xlsx' # "correct_abts" ---- A slice of the fmeca which requires abt corrections.
correct_dim_name = 'correct_dim.xlsx' # "correct_dim" --- A slice of fmeca which requires correct diminution levels


# -- File path
save_location = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets/AN_working/ProcessedData'

In [411]:
df.to_excel(os.path.join(save_location,extended_df_name),index=False)
reqCalc.to_excel(os.path.join(save_location,req_calc_name),index=False)
correct_abts.to_excel(os.path.join(save_location,correct_abts_name),index=False)
correct_dim.to_excel(os.path.join(save_location,correct_dim_name),index=False)